# Compression of Moist Air

In [ ]:
from thermostate import State, Q_, units, SystemInternational as SI

## Definitions

In [ ]:
T_1 = Q_(50.0, 'degC')
p_1 = Q_(0.9, 'bar')
phi_1 = Q_(0.7, 'dimensionless')
Vdot_1 = Q_(0.8, 'm**3/s')

T_2 = Q_(195.0, 'degC')
p_2 = Q_(1.5, 'bar')

## Problem Statement

Moist air enters a compressor operating at steady state at 50 degC, 0.9, bar, with a relative humidity of 70% and a volumetric flow rate of 0.8 m3/s. The moist air exits the compressor at 195 degC and 1.5 bar. Assuming the compressor is well insulated, determine

1. the relative humidity at the exit
2. the power input, in kW
3. the rate of entropy destruction, in kW/K

## Solution

### 1.  the relative humidity at the exit

The first thing we have to do is check that no water will be condensed at the outlet of the compressor. To do so, we compare the partial pressure of the water vapor at the outlet to the saturation pressure at the outlet temperature.

In [ ]:
st_2g = State('water', T=T_2, x=Q_(1.0, 'dimensionless'))
pg_2 = st_2g.p.to('bar')
print(pg_2)

The saturation pressure of the water vapor at the outlet is nearly 14 bar. The total pressure at the outlet is actually 1.5 bar (given in the problem statement), and the partial pressure of the water vapor is some fraction of that, so there definitely won't be any condensation at the outlet.

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

Since there is no condensation, this is a constant composition process and $\omega_1 = \omega_2$. If we can calculate the humidity ratio at the inlet, we can use it to find the partial pressure of the water vapor at the outlet, and determine the relative humidity. The humidity ratio is
$$\omega = 0.622 \frac{p_v}{p - p_v}$$
so at state 1

In [ ]:
st_1g = State('water', T=T_1, x=Q_(1.0, 'dimensionless'))
pg_1 = st_1g.p.to('bar')
pv_1 = phi_1*pg_1
print(pv_1)

In [ ]:
MW_air = Q_(28.97, 'kg/kmol')
MW_h2o = Q_(18.02, 'kg/kmol')
omega_1 = (MW_h2o/MW_air*pv_1/(p_1 - pv_1)).to('dimensionless')
print(omega_1)

With the value for the humidity ratio, we can find the partial pressure of the water vapor at state 2

In [ ]:
omega_2 = omega_1
pv_2 = (p_2*omega_2/(MW_h2o/MW_air + omega_2)).to('bar')
print(pv_2)

and finally, using this, we can find the relative humidity at state 2

In [ ]:
phi_2 = (pv_2/pg_2).to('dimensionless')
print(phi_2, phi_2.to('percent'))

### 2. the work input to the compressor

To find the work input to the compressor, we draw a control volume around the compressor and write the energy rate balance
\begin{align}
0 &= \dot{Q} - \dot{W} + \dot{m}_a h_{a1} + \dot{m}_v h_{v1} - \dot{m}_a h_{a2} - \dot{m}_v h_{v2} \\
\dot{W} &= \dot{m}_a (h_{a1} - h_{a2}) + \dot{m}_v (h_{v1} - h_{v2}) \\
\dot{W} &= \dot{m}_a [h_{a1} - h_{a2} + \omega(h_{v1} - h_{v2})]
\end{align}

$\dot{m}_a$ can be found by dividing the volumetric flow rate by the specific volume of the air at state 1
$$\dot{m}_a = \frac{\dot{V}_1}{v_{a1}}$$

In [ ]:
pa_1 = p_1 - pv_1
va_1 = ((units.molar_gas_constant/MW_air)*T_1/pa_1).to('m**3/kg')
mdot_a = Vdot_1/va_1
print(mdot_a)

Now to find the enthalpy values from `State` objects

In [ ]:
st_a1 = State('air', T=T_1, p=pa_1)
ha_1 = st_a1.h.to(SI.h)

st_a2 = State('air', T=T_2, p=(p_2 - pv_2))
ha_2 = st_a2.h.to(SI.h)

st_v1 = State('water', T=T_1, p=pv_1)
hv_1 = st_v1.h.to(SI.h)

st_v2 = State('water', T=T_2, p=pv_2)
hv_2= st_v2.h.to(SI.h)

Wdot_c = (mdot_a*(ha_1 - ha_2 + omega_1*(hv_1 - hv_2))).to('kW')
print(Wdot_c)

We can check what the answer would be if we used saturated vapor enthalpies for the water vapor

In [ ]:
st_v1 = State('water', T=T_1, x=Q_(1.0, 'dimensionless'))
hv_1 = st_v1.h.to(SI.h)

st_v2 = State('water', T=T_2, x=Q_(1.0, 'dimensionless'))
hv_2 = st_v2.h.to(SI.h)

Wdot_c = (mdot_a*(ha_1 - ha_2 + omega_1*(hv_1 - hv_2))).to('kW')
print(Wdot_c)

The answer is a little bit less than 4% different - pretty good, considering!

### 3. The rate of entropy production

Similar to the energy rate balance, the entropy rate balance reduces to
$$\dot{\sigma} = \dot{m}_a[s_{a2} - s_{a1} + \omega(s_{v2} - s_{v1})]$$

Redefining the states back to the partial pressures,

In [ ]:
st_a1 = State('air', T=T_1, p=pa_1)
sa_1 = st_a1.s.to(SI.s)

st_a2 = State('air', T=T_2, p=(p_2 - pv_2))
sa_2 = st_a2.s.to(SI.s)

st_v1 = State('water', T=T_1, p=pv_1)
sv_1 = st_v1.s.to(SI.s)

st_v2 = State('water', T=T_2, p=pv_2)
sv_2= st_v2.s.to(SI.s)

sigma_dot = mdot_a*(sa_2 - sa_1 + omega_1*(sv_2 - sv_1))
print(sigma_dot)

The rate of entropy production is positive, indicating that this process is irreversible.